In [1]:
with open('4_complete.gml', 'r') as file:
    lines = file.readlines()

edges = {}
current_edge = None

def read_x_y(line):
    # e.g., line = '\t\t\t\tpoint [ x 230.0000000000 y 540.0000000000 ]\n'
    # Extract x and y values without using re
    start_index = line.find('x') + 1
    end_index = line.find('y')
    x_str = line[start_index:end_index].strip()
    y_str = line[end_index + 1 : -3].strip()  # -3 to exclude the closing bracket and newline
    # Convert to float
    x = float(x_str)
    y = float(y_str)
    return(x, y)

for line in lines:
    if 'edge' in line:
        current_edge = []
        source=-1
        dest=-1
    elif 'source' in line and current_edge is not None:
        source = int(line.split()[1])
    elif 'target' in line and current_edge is not None:
        dest = int(line.split()[1])
    elif (']' in line) and (current_edge is not None) and (not ('point' in line)):
        edges[(source, dest)]=current_edge
        current_edge = None
    elif 'point' in line and current_edge is not None:
        # Extract coordinates of points in the line segment
        coordinates = read_x_y(line)
        current_edge.append( coordinates)


In [2]:
edges

{(0, 1): [(230.0, 540.0), (120.0, 540.0), (120.0, 430.0)],
 (0, 2): [(330.0, 440.0), (330.0, 430.0)],
 (0, 3): [(430.0, 540.0), (440.0, 540.0), (440.0, 120.0), (430.0, 120.0)],
 (1, 2): [(220.0, 330.0), (230.0, 330.0)],
 (1, 3): [(120.0, 230.0), (120.0, 120.0), (230.0, 120.0)],
 (2, 3): [(330.0, 230.0), (330.0, 220.0)]}

calculate waveguide lengths

In [3]:
def calculate_edge_length(points):
    length = 0
    for i in range(len(points) - 1):
        x1, y1 = points[i][0], points[i][1]
        x2, y2 = points[i + 1][0], points[i + 1][1]
        length += abs(x2 - x1) + abs(y2 - y1)
    return length

edge_lengths={}
for s_d, edge in edges.items():
    length = calculate_edge_length(edge)
    edge_lengths[s_d]=length
    print(f"Edge from {s_d[0]} to {s_d[1]} has length: {length}")

Edge from 0 to 1 has length: 220.0
Edge from 0 to 2 has length: 10.0
Edge from 0 to 3 has length: 440.0
Edge from 1 to 2 has length: 10.0
Edge from 1 to 3 has length: 220.0
Edge from 2 to 3 has length: 10.0


In [4]:
Tile_length = 200.0
Tile_real_length = 30 #[mm]
ratio=Tile_real_length/Tile_length
for s_d in edge_lengths.keys():
    edge_lengths[s_d]*=ratio 
    # now the unit of edge lengths is [mm]

In [5]:
edge_lengths

{(0, 1): 33.0,
 (0, 2): 1.5,
 (0, 3): 66.0,
 (1, 2): 1.5,
 (1, 3): 33.0,
 (2, 3): 1.5}

calculate crossings on each waveguide

In [6]:
from typing import Dict, List, Tuple

def count_crossings(edges: Dict[Tuple[int, int], List[Tuple[float, float]]]) -> Dict[Tuple[int, int], int]:
    crossings_count = {}

    # Function to check if two line segments intersect
    def do_segments_intersect(x1, y1, x2, y2, x3, y3, x4, y4):
        """
        Determine if two line segments intersect, given their endpoints (x1, y1), (x2, y2) and (x3, y3), (x4, y4).
        """
        # calculate the direction vectors for each segment
        dx1 = x2 - x1
        dy1 = y2 - y1
        dx2 = x4 - x3
        dy2 = y4 - y3
        # calculate the determinant
        det = dx1*dy2 - dx2*dy1
        # check if the determinant is zero
        if det == 0:
            return False
        # calculate the inverse determinant
        inv_det = 1 / det
        # calculate the difference vectors for each segment
        dx3 = x1 - x3
        dy3 = y1 - y3
        # calculate the parameters of the intersection point
        t1 = (dx2*dy3 - dx3*dy2) * inv_det
        t2 = (dx1*dy3 - dx3*dy1) * inv_det
        # check if the intersection point lies within both segments
        if 0 < t1 < 1 and 0 < t2 < 1:
            return True
        else:
            return False

    # Function to count crossings on a segment with all other segments
    def count_crossings_on_segment(edge, segment_idx):
        # source, destination = edge
        segment = edges[edge][segment_idx: segment_idx+2]
        crossings = 0

        for other_edge, other_segments in edges.items():
            if edge != other_edge:
                for other_segment_idx in range(len(other_segments) - 1):
                    if do_segments_intersect(edges[edge][segment_idx][0], edges[edge][segment_idx][1], 
                            edges[edge][segment_idx+1][0], edges[edge][segment_idx+1][1], 
                            edges[other_edge][other_segment_idx][0], edges[other_edge][other_segment_idx][1], 
                            edges[other_edge][other_segment_idx+1][0], edges[other_edge][other_segment_idx+1][1]):
                        crossings += 1
        return crossings

    # Iterate through each edge and count crossings
    for edge, waypoints in edges.items():
        crossings_on_edge = 0

        for segment_idx in range(len(waypoints) - 1):
            crossings_on_edge += count_crossings_on_segment(edge, segment_idx)

        crossings_count[edge] = crossings_on_edge

    return crossings_count

result = count_crossings(edges)
print(result)


{(0, 1): 0, (0, 2): 0, (0, 3): 0, (1, 2): 0, (1, 3): 0, (2, 3): 0}


In [11]:
edges={(0, 1): [(230.0, 540.0), (120.0, 540.0), (120.0, 430.0)],
 (0, 2): [(330.0, 440.0), (330.0, 430.0)],
 (0, 3): [(430.0, 540.0), (440.0, 540.0), (440.0, 120.0), (430.0, 120.0)],
 (1, 2): [(220.0, 330.0), (441.0, 330.0)],
 (1, 3): [(120.0, 230.0), (120.0, 120.0), (230.0, 120.0)],
 (2, 3): [(330.0, 230.0), (330.0, 220.0)]}

result = count_crossings(edges)
print(result)

{(0, 1): 0, (0, 2): 0, (0, 3): 1, (1, 2): 1, (1, 3): 0, (2, 3): 0}
